In [16]:
test_df = pd.concat([X_test.reset_index(), pd.Series(classifier.predict(X_test), name = 'Pred')], axis= 1)

NameError: name 'classifier' is not defined

In [ ]:
test_df.loc[test_df['Pred'] == 1]

In [ ]:
df.iloc[test_df.loc[test_df['Pred'] == 1]['index'].values]

In [ ]:
test_df['True_Pred'] = y_test.values

In [ ]:
test_df

In [ ]:
strify under sampling